In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "llama3"

In [33]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model = Ollama(model = MODEL)
embeddings = OllamaEmbeddings(model = MODEL)

In [44]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("book.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'book.pdf', 'page': 0, 'page_label': '1'}, page_content='|  1  \nGet\n4.0'),
 Document(metadata={'source': 'book.pdf', 'page': 1, 'page_label': '2'}, page_content='|  2'),
 Document(metadata={'source': 'book.pdf', 'page': 2, 'page_label': '3'}, page_content='|  3  \nDISCLAIMER\nBe informed that this book is solely written based on my personal experience and I, in no way claim, that \nthe information in it is 100% verified, as there are more and more researches coming every day, discretion is \nadvised. Furthermore, nothing contained herein is to be construed as Medical Advice. Use of any supplements/\ndrugs and exercise regimen should only be done under the directions and auspices of a licensed physician. \nThe writer does not claim to be a medical doctor nor does he purport to issue medical advice.\nFirst of all, let me congratulate you.\nYou wouldn’t have taken time to start reading it if you were not ready to change. \nThis shows you’re ready to begin y

In [45]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you don't know the answer, just say "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context = "Here is some context", question = "Here is a question"))


Answer the question based on the context below. If you don't know the answer, just say "I don't know".

Context: Here is some context

Question: Here is a question



In [46]:
chain =  prompt | model

In [47]:
chain.input_schema.schema()

/var/folders/nk/dfv879r135qf1g11chl_y8v80000gn/T/ipykernel_85905/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [48]:
chain.invoke({
    "context": "Abhiraj is 21 years old.",
    "question": "How old is Abhiraj ?"
})

'Abhiraj is 21 years old.'

In [49]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [53]:
retriever = vectorstore.as_retriever()

retriever.invoke("fitness")

[Document(metadata={'source': 'book.pdf', 'page': 0, 'page_label': '1'}, page_content='|  1  \nGet\n4.0'),
 Document(metadata={'source': 'book.pdf', 'page': 1, 'page_label': '2'}, page_content='|  2'),
 Document(metadata={'source': 'book.pdf', 'page': 47, 'page_label': '48'}, page_content='|  48'),
 Document(metadata={'source': 'book.pdf', 'page': 16, 'page_label': '17'}, page_content='|  17  \nAlternatively you can use Harris-Benedict Equation. Once you know your BMR, you can \ncalculate your Daily Calorie Needs based on your activity level. To determine your total daily \ncalorie needs, multiply your BMR by the appropriate activity factor, as follows:\nCalorie-Calculation \nFor sedentary (little or no exercise)\n= BMR x 1.2\nFor lightly active (light exercise/sports 1-3 days/ week)\n= BMR x 1.375\nFor moderately active (moderate exercise/sports 3-5days/week)\n = BMR x 1.55\nFor very active (hard exercise/sports 6-7 days a week)\n = BMR x 1.725\nTotal Calorie Needs Example\nIf you are

In [51]:
from operator import itemgetter

chain = (
    {
        "context" : itemgetter("question") | retriever,
        "question" : itemgetter("question")
    }
    | prompt
    | model
)

In [55]:
# chain.invoke({"question" : "Give me a short sumary on the book?"})

import pyttsx3

answer = chain.invoke({"question": "Give me a short summary on the book?"})

engine = pyttsx3.init()

engine.setProperty('rate', 150)
engine.setProperty('volume', 0.9) 

engine.say(answer)
engine.runAndWait()

KeyboardInterrupt: 